In [1]:
import os
import time
import datetime
from collections import namedtuple
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import schedule
import tinvest

In [71]:
import sys
sys.path.append("..")
from stats import get_stats_for_period

In [72]:
import telegram
TINKOFF_OPEN_API_TOKEN = "OPEN_API_TOKEN"
TELEGRAM_BOT_TOKEN = "TELEGRAM_BOT_TOKEN"

In [74]:
bot = telegram.Bot(token=os.getenv(TELEGRAM_BOT_TOKEN))
client = tinvest.SyncClient(os.getenv(TINKOFF_OPEN_API_TOKEN))

In [75]:
all_figi = client.get_market_stocks()

In [80]:
len(all_figi.payload.instruments)

1860

In [68]:
days_to_analyze = 60
asset = namedtuple("Asset", ["name", "figi", "currency", "closed"])
ts_value = namedtuple("TsValue", ["dttm", "value"])

date_to = datetime.datetime.now()
date_from = date_to - datetime.timedelta(days=days_to_analyze)

all_figi = client.get_market_etfs()
assets = list(map(lambda i: asset(i.name, i.figi, i.currency.name, []), (all_figi.payload.instruments)))

for a in assets:
    a.closed.extend(
        list(map(
            lambda x: ts_value(x.time.date(), float(x.c)),
            client.get_market_candles(
                a.figi,
                from_=date_from,
                to=date_to,
                interval=tinvest.schemas.CandleResolution.day
            ).payload.candles
        ))
    )

In [ ]:
assets_df = pd.DataFrame(assets).dropna()
assets_df = assets_df[assets_df.closed.apply(len) > 1]
assets_df = assets_df.explode('closed')
assets_df = assets_df.reset_index(drop=True)

assets_df['dt'] = assets_df['closed'].apply(lambda x: x.dttm if not pd.isnull(x) else x)
assets_df['closed'] = assets_df['closed'].apply(lambda x: x.value if not pd.isnull(x) else x)

unique_figi = assets_df.figi.drop_duplicates()
for figi in unique_figi:
    series_closed = assets_df[assets_df.figi == figi]['closed']

    assets_df.loc[assets_df.figi == figi, 'increase_rate'] = (
        series_closed / series_closed.shift()
    ).fillna(1)

    assets_df.loc[assets_df.figi == figi, 'total_increase'] = (
        assets_df.loc[assets_df.figi == figi, 'increase_rate']
        .cumprod()
        .tail(1)
        .reset_index(drop=True)
        [0]
    )

    grouped_by_figi_stats = assets_df.groupby(["figi", "name", "currency"], as_index=False).agg(
        std_increase_rate=('increase_rate', 'std'),
        total_increase=('total_increase', 'max'),
        count=('dt', 'count'),
    )

In [64]:
grouped_by_figi_stats

,figi,name,currency,std_increase_rate,total_increase,count
0,BBG000000000,Тинькофф Вечный портфель USD,usd,0.004348,1.027523,41
1,BBG000000001,Тинькофф Вечный портфель RUB,rub,0.004086,0.967895,41
2,BBG000000002,Тинькофф Вечный портфель EUR,eur,0.004345,1.033238,41
3,BBG004HNZFH2,FinEx Еврооблигации рос. компаний (RUB),rub,0.002414,0.994178,41
4,BBG005DXDPK9,FinEx Золото,rub,0.009927,1.049941,41
...,...,...,...,...,...,...
94,TCS00A103VL1,Тинькофф BuyBack Leaders,usd,0.008896,0.958376,22
95,TCS00A103VM9,Тинькофф ESG Leaders,usd,0.008464,0.999003,27
96,TCS00A103VN7,Тинькофф CyberSecurity,usd,0.012217,0.935323,27
97,TCS0BMDKNM37,FinEx Акций компаний развитых стран без США,rub,0.011006,1.056530,41


In [63]:
assets_df

,name,figi,currency,closed,dt,increase_rate,total_increase
0,Тинькофф iMOEX,BBG333333333,rub,7.492,2021-10-11,1.000000,0.900694
1,Тинькофф iMOEX,BBG333333333,rub,7.458,2021-10-12,0.995462,0.900694
2,Тинькофф iMOEX,BBG333333333,rub,7.416,2021-10-13,0.994368,0.900694
3,Тинькофф iMOEX,BBG333333333,rub,7.454,2021-10-14,1.005124,0.900694
4,Тинькофф iMOEX,BBG333333333,rub,7.452,2021-10-15,0.999732,0.900694
...,...,...,...,...,...,...,...
3772,Сбер - Взвешенный смарт фонд,BBG011C8J503,rub,10.256,2021-12-01,1.011839,0.946885
3773,Сбер - Взвешенный смарт фонд,BBG011C8J503,rub,10.209,2021-12-02,0.995417,0.946885
3774,Сбер - Взвешенный смарт фонд,BBG011C8J503,rub,10.214,2021-12-03,1.000490,0.946885
3775,Сбер - Взвешенный смарт фонд,BBG011C8J503,rub,10.108,2021-12-06,0.989622,0.946885


In [7]:
SLEEP_TIME_SECONDS = 60
DATE_FROM = datetime.datetime(2020, 11, 1)
DATE_TO = datetime.datetime(2021, 11, 1)

pbar = tqdm.tqdm(enumerate(assets), total=len(assets), leave=False)
pbar.set_description("Asset")
for i, a in pbar:
    a.closed.extend(
        list(map(
            lambda x: ts_value(x.time.date(), float(x.c)),
            client.get_market_candles(
                a.figi,
                from_=DATE_FROM,
                to=DATE_TO,
                interval=tinvest.schemas.CandleResolution.day
            ).payload.candles
        ))
    )
    
#     if i % 90 == 0 and i >= 90:
#         time.sleep(SLEEP_TIME_SECONDS)

In [443]:
# etf = assets
# share = assets

In [23]:
DAYS = 35

assets_df = pd.DataFrame(assets).dropna()
assets_df = assets_df[assets_df.closed.apply(len) > 1]
assets_df = assets_df.explode('closed')
assets_df = assets_df.reset_index(drop=True)

assets_df['dt'] = assets_df['closed'].apply(lambda x: x.dttm if not pd.isnull(x) else x)
assets_df['closed'] = assets_df['closed'].apply(lambda x: x.value if not pd.isnull(x) else x)
assets_df = assets_df[assets_df.dt >= (DATE_TO.date() - datetime.timedelta(days=DAYS))]

In [25]:
# assets_df.to_csv("etf.csv", index=False)

In [26]:
unique_figi = assets_df.figi.drop_duplicates()
pbar = tqdm.tqdm(unique_figi, total=len(unique_figi), leave=False)
pbar.set_description("FIGI")
for figi in pbar:
    series_closed = assets_df[assets_df.figi == figi]['closed']
    
    assets_df.loc[assets_df.figi == figi, 'increase_rate'] = (
        series_closed / series_closed.shift()
    ).fillna(1)
    
    assets_df.loc[assets_df.figi == figi, 'total_increase'] = (
        assets_df.loc[assets_df.figi == figi, 'increase_rate']
        .cumprod()
        .tail(1)
        .reset_index(drop=True)
        [0]
    )

In [27]:
tmp = assets_df.groupby(["figi", "name", "currency"], as_index=False).agg(
    std_increase_rate=('increase_rate', 'std'),
    total_increase=('total_increase', 'max'),
    count=('dt', 'count'),
)

## 1) min std, max increase

In [28]:
(
    tmp[tmp.currency == 'rub']
    .sort_values('std_increase_rate')
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
16,BBG00M8C8Y03,FinEx Акции казахстанских компаний,rub,0.009151,1.090909,25
67,TCS00A102XQ8,АТОН Золотодобытчики,rub,0.013383,1.034965,25
34,BBG00V0YCTS9,Райффайзен Индекс МосБиржи,rub,0.007417,1.033159,25
37,BBG00WJSKRX9,РСХБ - Вектор Устойчивого развития,rub,0.009077,1.025388,25
77,TCS00A102Y09,АТОН Серебро,rub,0.015198,1.022222,25
60,TCS00A101EJ5,ВТБ Индекс Мосбиржи,rub,0.008834,1.020446,25
58,BBG333333333,Тинькофф iMOEX,rub,0.008452,1.019641,25
68,TCS00A102XR6,АТОН Инновации,rub,0.018522,1.019495,25
14,BBG00M0C8YM7,Сбер Индекс Мосбиржи,rub,0.008952,1.019466,25
11,BBG0073DLHS1,FinEx Акции российских компаний,rub,0.009970,1.018243,23


In [29]:
(
    tmp[tmp.currency == 'usd']
    .sort_values('std_increase_rate')
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
84,TCS00A1039P6,Тинькофф Green Economy,usd,0.013879,1.120087,25
20,BBG00NNQMD85,Альфа-Капитал Технологии 100,usd,0.011925,1.039604,25
23,BBG00P5M77Y0,Альфа-Капитал S&P 500,usd,0.008686,1.039344,25
64,TCS00A102EQ8,Тинькофф S&P 500,usd,0.010180,1.033539,25
63,TCS00A102EM7,Тинькофф Индекс IPO,usd,0.018145,1.033303,25
24,BBG00PGVPH30,ВТБ Акции компаний США,usd,0.008822,1.028627,25
39,BBG00Y6D0N45,FinEx Акции компаний IT-сектора США,usd,0.010906,1.025172,25
56,BBG111111111,Тинькофф NASDAQ,usd,0.014359,1.023336,25
57,BBG222222222,Тинькофф Золото,usd,0.006848,1.018031,25
0,BBG000000000,Тинькофф Вечный портфель USD,usd,0.004867,1.015497,25


## 2) max std

In [30]:
(
    tmp[tmp.currency == 'rub']
    .sort_values('std_increase_rate', ascending=False)
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
16,BBG00M8C8Y03,FinEx Акции казахстанских компаний,rub,0.009151,1.090909,25
67,TCS00A102XQ8,АТОН Золотодобытчики,rub,0.013383,1.034965,25
34,BBG00V0YCTS9,Райффайзен Индекс МосБиржи,rub,0.007417,1.033159,25
37,BBG00WJSKRX9,РСХБ - Вектор Устойчивого развития,rub,0.009077,1.025388,25
77,TCS00A102Y09,АТОН Серебро,rub,0.015198,1.022222,25
60,TCS00A101EJ5,ВТБ Индекс Мосбиржи,rub,0.008834,1.020446,25
58,BBG333333333,Тинькофф iMOEX,rub,0.008452,1.019641,25
68,TCS00A102XR6,АТОН Инновации,rub,0.018522,1.019495,25
14,BBG00M0C8YM7,Сбер Индекс Мосбиржи,rub,0.008952,1.019466,25
11,BBG0073DLHS1,FinEx Акции российских компаний,rub,0.009970,1.018243,23


In [31]:
(
    tmp[tmp.currency == 'usd']
    .sort_values('std_increase_rate', ascending=False)
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
84,TCS00A1039P6,Тинькофф Green Economy,usd,0.013879,1.120087,25
20,BBG00NNQMD85,Альфа-Капитал Технологии 100,usd,0.011925,1.039604,25
23,BBG00P5M77Y0,Альфа-Капитал S&P 500,usd,0.008686,1.039344,25
64,TCS00A102EQ8,Тинькофф S&P 500,usd,0.010180,1.033539,25
63,TCS00A102EM7,Тинькофф Индекс IPO,usd,0.018145,1.033303,25
24,BBG00PGVPH30,ВТБ Акции компаний США,usd,0.008822,1.028627,25
39,BBG00Y6D0N45,FinEx Акции компаний IT-сектора США,usd,0.010906,1.025172,25
56,BBG111111111,Тинькофф NASDAQ,usd,0.014359,1.023336,25
57,BBG222222222,Тинькофф Золото,usd,0.006848,1.018031,25
0,BBG000000000,Тинькофф Вечный портфель USD,usd,0.004867,1.015497,25


## 3) max insrease

In [32]:
(
    tmp[tmp.currency == 'rub']
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
16,BBG00M8C8Y03,FinEx Акции казахстанских компаний,rub,0.009151,1.090909,25
67,TCS00A102XQ8,АТОН Золотодобытчики,rub,0.013383,1.034965,25
34,BBG00V0YCTS9,Райффайзен Индекс МосБиржи,rub,0.007417,1.033159,25
37,BBG00WJSKRX9,РСХБ - Вектор Устойчивого развития,rub,0.009077,1.025388,25
77,TCS00A102Y09,АТОН Серебро,rub,0.015198,1.022222,25
60,TCS00A101EJ5,ВТБ Индекс Мосбиржи,rub,0.008834,1.020446,25
58,BBG333333333,Тинькофф iMOEX,rub,0.008452,1.019641,25
68,TCS00A102XR6,АТОН Инновации,rub,0.018522,1.019495,25
14,BBG00M0C8YM7,Сбер Индекс Мосбиржи,rub,0.008952,1.019466,25
11,BBG0073DLHS1,FinEx Акции российских компаний,rub,0.009970,1.018243,23


In [33]:
(
    tmp[tmp.currency == 'usd']
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
84,TCS00A1039P6,Тинькофф Green Economy,usd,0.013879,1.120087,25
20,BBG00NNQMD85,Альфа-Капитал Технологии 100,usd,0.011925,1.039604,25
23,BBG00P5M77Y0,Альфа-Капитал S&P 500,usd,0.008686,1.039344,25
64,TCS00A102EQ8,Тинькофф S&P 500,usd,0.010180,1.033539,25
63,TCS00A102EM7,Тинькофф Индекс IPO,usd,0.018145,1.033303,25
24,BBG00PGVPH30,ВТБ Акции компаний США,usd,0.008822,1.028627,25
39,BBG00Y6D0N45,FinEx Акции компаний IT-сектора США,usd,0.010906,1.025172,25
56,BBG111111111,Тинькофф NASDAQ,usd,0.014359,1.023336,25
57,BBG222222222,Тинькофф Золото,usd,0.006848,1.018031,25
0,BBG000000000,Тинькофф Вечный портфель USD,usd,0.004867,1.015497,25
